# <div style="text-align: center"> <font size=+5> <ins>PREDICT ANSWERS from INDICATORS</ins> </font> </div>
___


### Imports and settings

In [9]:
#imports 

import numpy as np
import pandas as pd

import os

import ipywidgets as widgets



import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt



from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [2]:
#settings

pd.set_option('display.max_columns', 500)

plt.rcParams.update({'figure.dpi' : 300,
                     'mathtext.fontset' : 'stix', 
                     'font.family' : 'STIXGeneral'
                    })

___
## Get datasets

In [3]:
# get full dataset
data_issp = pd.read_pickle("Processed_Data/questionnaires/issp_questionnaires_common.pickle")

# convert values to numerical data
conv_dict = {
    "Respondent Data" : {
        "Sex" : {"Male" : -1, "Female" : 1},
        "Education level" : {"No degree" : 0, "Primary" : 1, "Secondary" : 2, "University" : 3},
        "Living area" : {"Rural area" : 0, "Small city" : 1, "Big city" : 2},
        "Political orientation" : {"Far Left" : -3, "Left" : -2, "Center Left" : -1, "Center" : 0, "None/Other" : 0, "Center Right" : 1, "Right" : 2, "Far Right" : 3},
        "Religious beliefs" : {"Atheist" : 0, "Christian" : 1, "Islamic" : 1, "Jewish" : 1, "Other" : 1}
    },
    "Questions" : {
        "Strongly Disagree" : -2,
        "Disagree" : -1,
        "Neither Agree nor Disagree" : 0,
        "Agree" : 1,
        "Strongly Agree" : 2,

        "Yes" : 1,
        "No" : -1,

        'Extremely dangerous': 2,
        'Very dangerous': 1,
        'Somewhat dangerous': 0,
        'Not very dangerous': -1,
        'Not dangerous at all': -2,

        "Always" : 3,
        "Often" : 2,
        "Sometimes" : 1,
        "Never" : 0,

        'Very willing': 2,
        'Fairly willing': 1,
        'Neither willing nor unwilling': 0,
        'Fairly unwilling': -1,
        'Very unwilling': -2
    }
}

#convert values to numerical
data_resp = data_issp.loc[:,("Respondent Data", slice(None))].apply(lambda col: col if col.name[1] not in conv_dict["Respondent Data"] else col.map(lambda x: np.nan if pd.isna(x) else conv_dict["Respondent Data"][col.name[1]][x]).astype(float))
data_quest = data_issp.loc[:,("Questions", slice(None))].map(lambda x: conv_dict["Questions"].get(x, x)).astype(float)

#cluster questions based on topic
clusters = {
    #TOPIC 1: awareness of the environmental consequences of societies’ modern, industrial activities
    "Awareness" : [
        'Danger to the environment (Air pollution by cars)',
        'Danger to the environment (Air pollution by industry)',
        'Danger to the environment (Pesticides and chemicals in farming)',
        'Danger to the environment (River, lake and stream pollution)',
        'Danger to the environment (Rise in the world’s temperature)',
        'Danger to the environment (Nuclear power stations)'
    ],

    #TOPIC 2: willingness to sacrifice personally in some manner (e.g. time, money) for the environment
    "Will to make sacrifices" : [
        'Willingness to Make Trade-Offs for Environment (Pay much higher prices)',
        'Willingness to Make Trade-Offs for Environment (Pay much higher taxes)',
        'Willingness to Make Trade-Offs for Environment (Cut your standard of living)',
        'Member of a group to preserve environment',
        'In the last five years, signed a petition',
        'In the last five years, given money',
        'In the last five years, participated in an environmental demonstration',
        'I do what is right even when it costs money and takes time'
    ],
    
    #TOPIC 3: importance of addressing environmental problems and of collective effort for resolving environmental
    #issues and how environmental issues inter- sect with economic issues, science and progress
    # NB!! answers are rescaled so that higher scores reflect pro-environment responses
    "Efficacy of environmental action" : [
        'We worry too much about harming environment',
        'We worry too much about future environment',
        'Science solves environmental problems',
        'Too difficult to do much about environment'
    ]
        
}

#rescale
data_to_cluster = data_quest.apply(lambda x: x if x.name[1] not in clusters["Efficacy of environmental action"] else -x)
data_quest_clustered = pd.concat([pd.concat([data_to_cluster.loc[:, ("Questions", clusters[topic])].sum(axis=1) for topic in clusters], axis=1, keys=clusters.keys())], keys=["Environmental concern dimension"],  names=["Variable Type", "Variable Name"], axis=1)


#get country-wide indicators columns
macro_indicators_dict = {(ind[:-7]) : pd.read_pickle(f"Processed_Data/indicators/{ind}") for ind in os.listdir("Processed_Data/indicators") if ind[-7:] == ".pickle"}
data_indic = pd.concat([pd.DataFrame.from_records([data_issp.index.map(lambda idx: indicator.loc[idx[1], idx[0]]) for indicator in macro_indicators_dict.values()], index=macro_indicators_dict.keys(), columns=data_issp.index).T], keys=["Country-wide indicators"], names=["Variable Type", "Variable Name"], axis=1)


#merge into single dataframes
data_issp_numerical = pd.concat([data_resp, data_indic, data_quest], axis=1) #separate questions
data_issp_numerical_clustered = pd.concat([data_resp, data_indic, data_quest_clustered], axis=1) #clustered questions


del(data_issp, data_resp, data_quest, data_to_cluster, data_quest_clustered, data_indic, conv_dict)


#dump
#data_issp_numerical.to_pickle("Saved_variables/full_dataset_numerical_quest+indic.pickle")
#data_issp_numerical_clustered.to_pickle("Saved_variables/full_dataset_numerical_clustered.pickle")

In [3]:
#quick load
data_issp_numerical = pd.read_pickle("Saved_variables/full_dataset_numerical_quest+indic.pickle")
data_issp_numerical_clustered = pd.read_pickle("Saved_variables/full_dataset_numerical_clustered.pickle")

___

In [4]:
#group by year and country (weighted mean)
data_issp_numerical_grouped = data_issp_numerical.drop(columns=("Respondent Data", "Weight")).mul(data_issp_numerical.loc[:,("Respondent Data", "Weight")], axis=0).groupby(["Year", "Country"]).sum(min_count=1).div(data_issp_numerical.loc[:,("Respondent Data", "Weight")].groupby(["Year", "Country"]).sum(), axis=0)
data_issp_numerical_clustered_grouped = data_issp_numerical_clustered.drop(columns=("Respondent Data", "Weight")).mul(data_issp_numerical_clustered.loc[:,("Respondent Data", "Weight")], axis=0).groupby(["Year", "Country"]).sum(min_count=1).div(data_issp_numerical_clustered.loc[:,("Respondent Data", "Weight")].groupby(["Year", "Country"]).sum(), axis=0)

In [5]:
#rescale so that dimensions columns are in range [-1,+1]
data_issp_numerical_clustered.loc[:,("Environmental concern dimension", slice(None))] /= [12,12,8]
data_issp_numerical_clustered_grouped.loc[:,("Environmental concern dimension", slice(None))] /= [12,12,8]

___
## Get relevant indicators

In [4]:
def fit_lasso(yX, alpha, weights=None):
    model = Lasso(alpha)
    model.fit(yX.iloc[:,1:], yX.iloc[:,0], sample_weight=weights)
    return {"alpha" : alpha, "loss" : 1-model.score(yX.iloc[:,1:], yX.iloc[:,0], sample_weight=weights), "coefficients" : model.coef_, "model" : model}

def plot_indicators(data, question, year, indicators):
    yX = data.loc[(year, slice(None)),[("Environmental concern dimension", question), *indicators]].dropna().astype(float)
    yX.iloc[:,:] = StandardScaler().fit_transform(yX.to_numpy())
    
    try:
        weights = data.loc[yX.index, ("Respondent Data", "Weight")]
    except:
        weights = None
    
    res = pd.DataFrame([fit_lasso(yX, alpha, weights) for alpha in np.logspace(-2,0,250)])
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(6.4*2,4.8), gridspec_kw={"width_ratios" : (5,95)}, sharey=True)
    fig.subplots_adjust(wspace=0)
    fig.suptitle(f"Most important indicators to predict '{question}' in {year}", fontsize=20)
    data = np.stack(res["coefficients"].to_numpy())
    sns.heatmap([[np.log(l)] for l in res["loss"]], ax=ax1, cbar=False)
    sns.heatmap(data, cmap="bwr", center=0, ax=ax2)
    ax2.set_title("Coefficients in LASSO regression")
    ax2.set_xticklabels([col[1] for col in yX.iloc[:,1:].columns], rotation=50, horizontalalignment="right", fontsize=8)
    ax2.set_xlabel("Indicator")

    for ax in [ax1]:
        ax.set_yticks(res.index[::50])
        ax.set_yticklabels([f"{l:.0e}" for l in res.loc[::50,"alpha"]], rotation=30, horizontalalignment="right", verticalalignment="top", fontsize=8)
        ax.tick_params(axis="y", pad=0)
        ax.set_ylabel("L1 regularization hyperparameter")
    
    ax1.set_title("Model error")
    ax1.set_xticks([])
    
    plt.tight_layout()

### Macro Indicators

In [6]:
@widgets.interact(year=[1993,2000,2010,2020], dimension=data_issp_numerical_clustered.loc[:,"Environmental concern dimension"].columns)
def plot(dimension, year=2020):
    indicators = [("Country-wide indicators", ind) for ind in ["GHG_emissions", "PM25_exposure", "GDP_growth", "gini_index", "GDP_pcap_ppp", "water_withdrawals", "pesticides_use"]]
    plot_indicators(data_issp_numerical_clustered_grouped, dimension, year, indicators)

interactive(children=(Dropdown(description='dimension', options=('Awareness', 'Will to make sacrifices', 'Effi…

### Individual indicators

In [7]:
@widgets.interact(year=[1993,2000,2010,2020], dimension=data_issp_numerical_clustered.loc[:,"Environmental concern dimension"].columns, all_countries=False, country=data_issp_numerical_clustered.index.get_level_values("Country").unique())
def plot(dimension, country, all_countries, year=2020):
    indicators = [("Respondent Data", ind) for ind in data_issp_numerical.loc[:, "Respondent Data"].drop(columns="Weight").columns]
    if all_countries:
        plot_indicators(data_issp_numerical_clustered, dimension, year, indicators)
    else:
        plot_indicators(data_issp_numerical_clustered.query("Country == @country"), dimension, year, indicators)

interactive(children=(Dropdown(description='dimension', options=('Awareness', 'Will to make sacrifices', 'Effi…

___
## Linear Fit

In [6]:
def fit_linear(yX, weights=None):
        
    #fill missing values with columns average
    yX = yX.fillna(yX.mean())
    #if whole column is missing fill with 0, then replace with NaN in output
    NaN_cols = yX.iloc[:,1:].mean().isna().to_numpy()
    yX = yX.fillna(0)

    #for large datasets: single train-test split
    if len(data) > 500:
        yX_train, yX_test = train_test_split(yX, test_size=0.2)

        if weights is not None:
            weights_train, weights_test = weights.loc[yX_train.index], weights.loc[yX_test.index]
        else:
            weights_train, weights_test = None, None
        
        #standardize data
        scaler = StandardScaler()
        yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
        yX_test.iloc[:,:] = scaler.transform(yX_test)
        
        #fit linear regression model
        model = LinearRegression(fit_intercept=False)
        model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

        #compute metrics
        yX_pred = pd.DataFrame().reindex_like(yX_test)
        yX_pred.iloc[:,0] = model.predict(yX_test.iloc[:,1:])
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)
        yX_test.iloc[:,:] = scaler.inverse_transform(yX_test)
        
        r2_loss = r2_score(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        mae = mean_absolute_error(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        

        return {"coefficients" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.coef_)], "losses" : [r2_loss, mae]}


    #for small datasets, leave one out cross validation
    else:
        
        yX_pred = pd.DataFrame().reindex_like(yX) #prepare dataframe for predictions
        
        for idx_num, idx_val in enumerate(yX.index):
            yX_train, yX_test = yX.loc[yX.index != idx_val, :], yX.loc[[idx_val], :]

            if weights is not None:
                weights_train = weights.loc[weights.index != idx_val]
            else:
                weights_train = None

            #standardize data
            scaler = StandardScaler()
            yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
            yX_test.iloc[:,:] = scaler.transform(yX_test)

            #fit linear regression model
            model = LinearRegression(fit_intercept=False)
            model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

            #get predictions
            yX_pred.iloc[idx_num,0] = model.predict(yX_test.iloc[:,1:])

        #rescale back to original predictions data
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)

        #compute metrics
        r2_loss = r2_score(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        mae = mean_absolute_error(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        
        return {"coefficients" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.coef_)], "losses" : [r2_loss, mae]}

In [ ]:
#get indicators dataframes


##############################################################################################################################################################################################
indicators_macro = [("Country-wide indicators", ind) for ind in ["GHG_emissions", "PM25_exposure", "GDP_growth", "gini_index", "GDP_pcap_ppp", "water_withdrawals", "pesticides_use"]]
predictive_indicators_macro_linearFit  = pd.DataFrame(columns=[("Indicators",ind) for _, ind in indicators_macro]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered_grouped.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):    
    data = data_issp_numerical_clustered_grouped.loc[(year, slice(None)), [dimension, *indicators_macro]]
    res = fit_linear(data)
    predictive_indicators_macro_linearFit [(dimension[1], year)] = [*res["coefficients"], *res["losses"]]

predictive_indicators_macro_linearFit = predictive_indicators_macro_linearFit.T
predictive_indicators_macro_linearFit.index = pd.MultiIndex.from_tuples(predictive_indicators_macro_linearFit.index, names=["Environmental concern dimension", "Year"])
predictive_indicators_macro_linearFit.columns = pd.MultiIndex.from_tuples(predictive_indicators_macro_linearFit.columns)

##############################################################################################################################################################################################
indicators_individual = [("Respondent Data", ind) for ind in data_issp_numerical.loc[:, "Respondent Data"].drop(columns="Weight").columns]
predictive_indicators_individual_linearFit = pd.DataFrame(columns=[("Indicators", ind) for _, ind in indicators_individual]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):
    for country in data_issp_numerical_clustered.loc[year].index.get_level_values("Country").unique():
        data = data_issp_numerical_clustered.loc[(year, country, slice(None)), [dimension, *indicators_individual]]
        weights = data_issp_numerical_clustered.loc[(year, country, slice(None)), ("Respondent Data", "Weight")]
        res = fit_linear(data, weights)
        predictive_indicators_individual_linearFit[(dimension[1], year, country)] = [*res["coefficients"], *res["losses"]]
        
predictive_indicators_individual_linearFit = predictive_indicators_individual_linearFit.T
predictive_indicators_individual_linearFit.index = pd.MultiIndex.from_tuples(predictive_indicators_individual_linearFit.index, names=["Environmental concern dimension", "Year", "Country"])
predictive_indicators_individual_linearFit.columns = pd.MultiIndex.from_tuples(predictive_indicators_individual_linearFit.columns)



##############################################################################################################################################################################################
#dump results

#predictive_indicators_macro_linearFit.to_pickle("./Saved_variables/predictive_indicators_macro_linearFit.pickle")
#predictive_indicators_individual_linearFit.to_pickle("./Saved_variables/predictive_indicators_individual_linearFit.pickle")

In [31]:
#quick load

predictive_indicators_macro_linearFit = pd.read_pickle("./Saved_variables/predictive_indicators_macro_linearFit.pickle")
predictive_indicators_individual_linearFit = pd.read_pickle("./Saved_variables/predictive_indicators_individual_linearFit.pickle")

### Plot results

In [32]:
def cluster_dataframe(data, method="ward", row_cluster=True, col_cluster=True):
    from scipy.cluster.hierarchy import linkage, dendrogram

    if row_cluster:
        row_linkage = linkage(data, method=method, optimal_ordering=True)
        row_order = dendrogram(row_linkage, no_plot=True)['leaves']
    else:
        row_order = slice(None)

    if col_cluster:
        col_linkage = linkage(data.T, method=method,  optimal_ordering=True)
        col_order = dendrogram(col_linkage, no_plot=True)['leaves']

    else:
        col_order = slice(None)
        
    abs_max = data.abs().max(axis=None)
    
    return data.iloc[row_order, col_order]

def style_dataframe(data):
    max = data.loc[:,"Indicators"].abs().max(axis=None)
    styled_df = data.style.background_gradient(cmap="bwr", axis=None, vmin=-max, vmax=max, subset="Indicators")
    
    cm = sns.light_palette("green", reverse=True, as_cmap=True)
    max = data.loc[:,("Loss", "r2_score")].max(axis=None)
    min = data.loc[:,("Loss", "r2_score")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "r2_score")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Loss", "mae")].max(axis=None)
    min = data.loc[:,("Loss", "mae")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "mae")], vmin=min, vmax=max)
    
    
    display(styled_df)

#### Individual indicators

In [33]:
data = predictive_indicators_individual_linearFit.groupby(["Environmental concern dimension", "Year"]).mean() #average over all countries
#data = predictive_indicators_individual_linearFit.query("Country == 'United States'") #select specific country
#data = predictive_indicators_individual_linearFit.loc[("Awareness",2010,slice(None)),:] #select topic and year
#data = predictive_indicators_individual_linearFit #all data

cluster_rows = False
cluster_columns = True

data_clustered = cluster_dataframe(data.fillna(0).loc[:, ("Indicators", slice(None))], row_cluster=cluster_rows, col_cluster=cluster_columns)
cols = [*data_clustered.columns, *data.loc[:,("Loss", slice(None))].columns]
rows = data_clustered.index
style_dataframe(data.loc[rows,cols])

**Conclusions:**

1. **Political orientation** more important in recent years -> more polarized, environmental issues become political issues (left -> more pro-environment)
2. **Education level** important in predicting Will to make sacrifices and Efficacy of env. action (more educ. -> more pro-environment)
3. **Sex** correlated with awareness (females -> more aware)
4. **Age** slight correlation with all env. concern dimensions (older -> less concerned)

#### Macro indicators

In [34]:
data = predictive_indicators_macro_linearFit
clustered_cols = [*cluster_dataframe(data.loc[:, ("Indicators", slice(None))], row_cluster=False).columns, *data.loc[:,("Loss", slice(None))].columns]
style_dataframe(data.loc[:,clustered_cols])

**Conclusions:**

1. **GDP** most important predictor -> less aware, but more willing to act (make sacrifices, believes in environmental action)

___
## Decision Trees

In [10]:
def fit_trees(yX, weights=None):
        
    #fill missing values with columns average
    yX = yX.fillna(yX.mean())
    #if whole column is missing fill with 0, then replace with NaN in output
    NaN_cols = yX.iloc[:,1:].mean().isna().to_numpy()
    yX = yX.fillna(0)

    #for large datasets: single train-test split
    if len(data) > 500:
        yX_train, yX_test = train_test_split(yX, test_size=0.2)

        if weights is not None:
            weights_train, weights_test = weights.loc[yX_train.index], weights.loc[yX_test.index]
        else:
            weights_train, weights_test = None, None
        
        #standardize data
        scaler = StandardScaler()
        yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
        yX_test.iloc[:,:] = scaler.transform(yX_test)
        
        #fit linear regression model
        model = GradientBoostingRegressor()
        model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

        #compute metrics
        yX_pred = pd.DataFrame().reindex_like(yX_test)
        yX_pred.iloc[:,0] = model.predict(yX_test.iloc[:,1:])
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)
        yX_test.iloc[:,:] = scaler.inverse_transform(yX_test)
        
        r2_loss = r2_score(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        mae = mean_absolute_error(yX_test.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights_test)
        

        return {"importance" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.feature_importances_)], "losses" : [r2_loss, mae]}


    #for small datasets, leave one out cross validation
    else:
        
        yX_pred = pd.DataFrame().reindex_like(yX) #prepare dataframe for predictions
        
        for idx_num, idx_val in enumerate(yX.index):
            yX_train, yX_test = yX.loc[yX.index != idx_val, :], yX.loc[[idx_val], :]

            if weights is not None:
                weights_train = weights.loc[weights.index != idx_val]
            else:
                weights_train = None

            #standardize data
            scaler = StandardScaler()
            yX_train.iloc[:,:] = scaler.fit_transform(yX_train)
            yX_test.iloc[:,:] = scaler.transform(yX_test)

            #fit linear regression model
            model = GradientBoostingRegressor()
            model.fit(yX_train.iloc[:, 1:], yX_train.iloc[:,0], sample_weight=weights_train)

            #get predictions
            yX_pred.iloc[idx_num,0] = model.predict(yX_test.iloc[:,1:])

        #rescale back to original predictions data
        yX_pred.iloc[:,:] = scaler.inverse_transform(yX_pred)

        #compute metrics
        r2_loss = r2_score(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        mae = mean_absolute_error(yX.iloc[:,0], yX_pred.iloc[:,0], sample_weight=weights)
        
        return {"importance" : [coef if NaN_cols[i] == False else np.nan for i,coef in enumerate(model.feature_importances_)], "losses" : [r2_loss, mae]}

In [ ]:
#get indicators dataframes


##############################################################################################################################################################################################
indicators_macro = [("Country-wide indicators", ind) for ind in ["GHG_emissions", "PM25_exposure", "GDP_growth", "gini_index", "GDP_pcap_ppp", "water_withdrawals", "pesticides_use"]]
predictive_indicators_macro_boostedTrees = pd.DataFrame(columns=[("Indicators",ind) for _, ind in indicators_macro]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered_grouped.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):    
    data = data_issp_numerical_clustered_grouped.loc[(year, slice(None)), [dimension, *indicators_macro]]
    res = fit_trees(data)
    predictive_indicators_macro_boostedTrees[(dimension[1], year)] = [*res["importance"], *res["losses"]]

predictive_indicators_macro_boostedTrees = predictive_indicators_macro_boostedTrees.T
predictive_indicators_macro_boostedTrees.index = pd.MultiIndex.from_tuples(predictive_indicators_macro_boostedTrees.index, names=["Environmental concern dimension", "Year"])
predictive_indicators_macro_boostedTrees.columns = pd.MultiIndex.from_tuples(predictive_indicators_macro_boostedTrees.columns)


##############################################################################################################################################################################################
indicators_individual = [("Respondent Data", ind) for ind in data_issp_numerical.loc[:, "Respondent Data"].drop(columns="Weight").columns]
predictive_indicators_individual_boostedTrees = pd.DataFrame(columns=[("Indicators", ind) for _, ind in indicators_individual]+[("Loss","r2_score"),("Loss", "mae")]).T

for year, dimension in ((y,d) for d in data_issp_numerical_clustered.loc[:,("Environmental concern dimension", slice(None))].columns for y in [1993,2000,2010,2020]):
    for country in data_issp_numerical_clustered.loc[year].index.get_level_values("Country").unique():
        data = data_issp_numerical_clustered.loc[(year, country, slice(None)), [dimension, *indicators_individual]]
        weights = data_issp_numerical_clustered.loc[(year, country, slice(None)), ("Respondent Data", "Weight")]
        res = fit_trees(data, weights)
        predictive_indicators_individual_boostedTrees[(dimension[1], year, country)] = [*res["importance"], *res["losses"]]
        
predictive_indicators_individual_boostedTrees = predictive_indicators_individual_boostedTrees.T
predictive_indicators_individual_boostedTrees.index = pd.MultiIndex.from_tuples(predictive_indicators_individual_boostedTrees.index, names=["Environmental concern dimension", "Year", "Country"])
predictive_indicators_individual_boostedTrees.columns = pd.MultiIndex.from_tuples(predictive_indicators_individual_boostedTrees.columns)



##############################################################################################################################################################################################
#dump results

#predictive_indicators_macro_boostedTrees.to_pickle("./Saved_variables/predictive_indicators_macro_boostedTrees.pickle")
#predictive_indicators_individual_boostedTrees.to_pickle("./Saved_variables/predictive_indicators_individual_boostedTrees.pickle")

In [27]:
#quick load

predictive_indicators_macro_boostedTrees = pd.read_pickle("./Saved_variables/predictive_indicators_macro_boostedTrees.pickle")
predictive_indicators_individual_boostedTrees = pd.read_pickle("./Saved_variables/predictive_indicators_individual_boostedTrees.pickle")

### Plot results

In [36]:
def cluster_dataframe(data, method="ward", row_cluster=True, col_cluster=True):
    from scipy.cluster.hierarchy import linkage, dendrogram

    if row_cluster:
        row_linkage = linkage(data, method=method, optimal_ordering=True)
        row_order = dendrogram(row_linkage, no_plot=True)['leaves']
    else:
        row_order = slice(None)

    if col_cluster:
        col_linkage = linkage(data.T, method=method,  optimal_ordering=True)
        col_order = dendrogram(col_linkage, no_plot=True)['leaves']

    else:
        col_order = slice(None)
        
    abs_max = data.abs().max(axis=None)
    
    return data.iloc[row_order, col_order]

def style_dataframe(data):
    max = data.loc[:,"Indicators"].max(axis=None)
    min = data.loc[:,"Indicators"].min(axis=None)
    styled_df = data.style.background_gradient(cmap="viridis", axis=None, vmin=min, vmax=max, subset="Indicators")
    
    cm = sns.light_palette("green", reverse=True, as_cmap=True)
    max = data.loc[:,("Loss", "r2_score")].max(axis=None)
    min = data.loc[:,("Loss", "r2_score")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "r2_score")], vmin=min, vmax=max)

    cm = sns.light_palette("green", reverse=False, as_cmap=True)
    max = data.loc[:,("Loss", "mae")].max(axis=None)
    min = data.loc[:,("Loss", "mae")].min(axis=None)
    styled_df = styled_df.background_gradient(cmap=cm, subset=[("Loss", "mae")], vmin=min, vmax=max)
    
    
    display(styled_df)

#### Individual indicators

In [37]:
data = predictive_indicators_individual_boostedTrees.groupby(["Environmental concern dimension", "Year"]).mean() #average over all countries
#data = predictive_indicators_individual_boostedTrees.query("Country == 'United States'") #select specific country
#data = predictive_indicators_individual_boostedTrees.loc[("Awareness",2010,slice(None)),:] #select topic and year
#data = predictive_indicators_individual_boostedTrees #all data

cluster_rows = False
cluster_columns = True

data_clustered = cluster_dataframe(data.fillna(0).loc[:, ("Indicators", slice(None))], row_cluster=cluster_rows, col_cluster=cluster_columns)
cols = [*data_clustered.columns, *data.loc[:,("Loss", slice(None))].columns]
rows = data_clustered.index
style_dataframe(data.loc[rows,cols])

#### Macro indicators

In [30]:
data = predictive_indicators_macro_boostedTrees
clustered_cols = [*cluster_dataframe(data.loc[:, ("Indicators", slice(None))], row_cluster=False).columns, *data.loc[:,("Loss", slice(None))].columns]
style_dataframe(data.loc[:,clustered_cols])